In [10]:
'''READ ME:

This is a Python Jupyter Notebook desiged to convert MnDOT sign inventory points into a dgn file with the proper signs 
at those locations. This script will print out any sign codes that did not match one found in the database. NOTE
this project needs to be run directly in ArcPro because the map's CRS needs to be changed along with the version
of the signs.

Sebastian Brauer -- 7/12/24
'''

import math

arcpy.env.workspace = r'C:\GIS_Scripting\SignProject\SignProject.gdb'

input_project_name = 'Twin_Harbors_TH_61' # REQUIRED USER INPUT The goal is to come up with a unique name, the city and TH make a good combo, feel free to add your last name as well.
input_county_name = "Lake" # REQUIRED USER INPUT in order to get the right projection. Please capitalize the first letter of the county, feel free to review the dictionary if needed.
input_csv_file = r'C:\GIS_Scripting\Test Data\SignsTest4Col.csv' # REQUIRED USER INPUT make sure to leave the 
input_save_location = r'C:\Users\sbrauer\Downloads' # REQUIRED USER INPUT output save location should be a folder

master_sign_file = 'SignDatabase_NOPROJ_Diss' # Reach out if you have a more updated sign database (7/12/24)
csv_projection = arcpy.SpatialReference(4152) # If you think that the csv is not projected in GCS then either reach out or change to the proper ESPG/WKID projection

mn_county_systems_epsg = {
    "Aitkin": 103700,
    "Anoka": 103708,
    "Becker": 103709,
    "Beltrami North": 103710,
    "Beltrami South": 103711,
    "Benton": 103712,
    "Big Stone": 103713,
    "Blue Earth": 103714,
    "Brown": 103715,
    "Carlton": 103716,
    "Carver": 103717,
    "Cass North": 103718,
    "Cass South": 103719,
    "Chippewa": 103720,
    "Chisago": 103721,
    "Clay": 103701,
    "Clearwater": 103702,
    "Cook North": 103722,
    "Cook South": 103723,
    "Cottonwood": 103724,
    "Crow Wing": 103725,
    "Dakota": 103726,
    "Dodge": 103727,
    "Douglas": 103728,
    "Faribault": 103729,
    "Fillmore": 103730,
    "Freeborn": 103731,
    "Goodhue": 103732,
    "Grant": 103733,
    "Hennepin": 103734,
    "Houston": 1037235,
    "Hubbard": 103703,
    "Isanti": 103736,
    "Itasca North": 103737,
    "Itasca South": 103738,
    "Jackson": 103739,
    "Kanabec": 103740,
    "Kandiyohi": 103741,
    "Kittson": 103742,
    "Koochiching": 103743,
    "Lac qui Parle": 103744,
    "Lake": 103704,
    "Lake of the Woods North": 103745,
    "Lake of the Woods South": 103746,
    "Le Sueur": 103747,
    "Lincoln": 103748,
    "Lyon": 103749,
    "Mahnomen": 103751,
    "Marshall": 103752,
    "Martin": 103753,
    "McLeod": 103750,
    "Meeker": 103754,
    "Mille Lacs": 103705,
    "Morrison": 103755,
    "Mower": 103756,
    "Murray": 103757,
    "Nicollet": 103758,
    "Nobles": 103759,
    "Norman": 103760,
    "Olmsted": 103761,
    "Ottertail": 103762,
    "Pennington": 103763,
    "Pine": 103764,
    "Pipestone": 103765,
    "Polk": 103766,
    "Pope": 103767,
    "Ramsey": 103768,
    "Red Lake": 103769,
    "Redwood": 103770,
    "Renville": 103771,
    "Rice": 103772,
    "Rock": 103773,
    "Roseau": 103774,
    "St. Louis": 103695,
    "St. Louis Central": 103776,
    "St. Louis North": 103775,
    "St. Louis South": 103777,
    "Scott": 103778,
    "Sherburne": 103779,
    "Sibley": 103780,
    "Stearns": 103781,
    "Steele": 103782,
    "Stevens": 103783,
    "Swift": 103784,
    "Todd": 103785,
    "Traverse": 103786,
    "Wabasha": 103787,
    "Wadena": 103788,
    "Waseca": 103789,
    "Washington": 103706,
    "Watonwan": 103790,
    "Wilkin": 103707,
    "Winona": 103791,
    "Wright": 103792,
    "Yellow Medicine": 103793
}

direction_dict = {
    'North': 180,
    'Northeast': 225,
    'East': 270,
    'Southeast': 315,
    'South': 0,
    'Southwest': 45,
    'West': 90,
    'Northwest': 135
}


epsg_code = mn_county_systems_epsg.get(input_county_name) # Use the input county name to get the WKID/ESPG
spatial_reference = arcpy.SpatialReference(epsg_code) # Create a Spatial Reference item with this projection WKID/ESPG


aprx = arcpy.mp.ArcGISProject('current')
map_obj = aprx.listMaps("Map")[0]
map_obj.spatialReference = spatial_reference # Set the new spatial reference to the map's default view
aprx.save() # Save the changes to the project


master_dgn_file = f"SignDatabase_{input_county_name}" # Check to see if input county already has projected signs
arcpy.management.CopyFeatures(master_sign_file, master_dgn_file)
arcpy.management.DefineProjection(master_dgn_file, spatial_reference)


temp_HARN_83_csv = f"{input_project_name}_csv_geo_projected" # Name of temp csv with 4152 ESPG
csv_input_class = f"{input_project_name}_csvPointProjected" # Name of points layer used for analysis
arcpy.management.XYTableToPoint(input_csv_file, temp_HARN_83_csv, "LONGITUDE", "LATITUDE", None, csv_projection) # Run Table to Point Tool
arcpy.management.Project(temp_HARN_83_csv, csv_input_class, spatial_reference) # Run the Project

arcpy.management.AddField(csv_input_class, "X", "DOUBLE") # Add X field
arcpy.management.AddField(csv_input_class, "Y", "DOUBLE") # Add Y field
arcpy.management.CalculateGeometryAttributes(csv_input_class, "X POINT_X;Y POINT_Y", "", "", spatial_reference) # Calculate Easting and Northing
arcpy.management.Delete(temp_HARN_83_csv)


# Create an empty feature class with the same schema as the master feature class -- NOTE to not copy the spatial reference from the master file because dgns come in with limited spatial domain
output_feature_class = f"{input_project_name}_signs"
arcpy.CreateFeatureclass_management(out_path = arcpy.env.workspace,
                                    out_name = output_feature_class,
                                    geometry_type = "POLYLINE",  # Assuming geometry type is polyline; change if needed
                                    template = master_dgn_file,
                                    spatial_reference = spatial_reference)
arcpy.management.AddField(output_feature_class, "Moved", "TEXT")

    
# Function to move features to a new location based on centroid
def move_features_to_location(layer, new_lat, new_long, sign_code):
    with arcpy.da.UpdateCursor(layer, ["SHAPE@XY", "CadModel", "Moved"]) as cursor:
        for row in cursor: 
            row[0] = (new_long, new_lat)
            row[2] = "Y"
            model = row[1]
            if model == 'Design':
                row[1] = sign_code + ' - Design'
            elif model == 'NoCode':
                row[1] = sign_code + ' - No Match'
            cursor.updateRow(row)
            
def rotatepoint(point, pivotpoint, angle):
    angle_rad = -math.radians(angle)
    ox, oy = pivotpoint.X, pivotpoint.Y
    px, py = point.X, point.Y
    qx = ox + math.cos(angle_rad) * (px - ox) - math.sin(angle_rad) * (py - oy)
    qy = oy + math.sin(angle_rad) * (px - ox) + math.cos(angle_rad) * (py - oy)    
    return arcpy.Point(qx,qy)

def rotate(layer, direction):
    with arcpy.da.UpdateCursor(layer, ['SHAPE@']) as cursor:
        for row in cursor:
            polylist = []
            for part in row[0]:
                partlist = []
                for pnt in part:
                    if pnt is not None: #Polygons with inner rings will have None pnt(s) which can be skipped
                        partlist.append(rotatepoint(pnt, row[0].centroid, direction)) #Centroid is pivot point
                        polylist.append(partlist)
            row[0] = arcpy.Polyline(arcpy.Array(polylist))
            cursor.updateRow(row)

# Read the input feature class and process each row
with arcpy.da.SearchCursor(csv_input_class, ["X", "Y", "SIGN_CODE", "SIGN_CLASS", "PANEL_FACI", "SIGN_STATU"]) as cursor:
    for row in cursor:
        x = row[0]
        y = row[1]
        sign_code = row[2]
        sign_class = row[3]
        direction = row[4]
        sign_status = row[5]

        if sign_status == 'Retired':
            # Add a symbol to say there is a design sign here.
            continue
            
        if sign_class == 'Design': # use the square x
            old_sign_code = sign_code
            sign_code = 'Design'
        
        escaped_sign_code = sign_code.replace("'", "''")
        where_clause = f"CadModel = '{escaped_sign_code}'"
        arcpy.management.SelectLayerByAttribute(master_dgn_file, "NEW_SELECTION", where_clause)
        
        count = int(arcpy.GetCount_management(master_dgn_file).getOutput(0))
        if count == 0:
            print(f"No features found for SIGN_CODE: {sign_code}. Using default no find symbol.")
            old_sign_code = sign_code
            sign_code = 'NoCode'
            where_clause = f"CadModel = '{sign_code}'"
            arcpy.management.SelectLayerByAttribute(master_dgn_file, "NEW_SELECTION", where_clause)
        
        arcpy.Append_management(master_dgn_file, output_feature_class, "NO_TEST")
        where_clause = f"CadModel = '{sign_code}' And Moved IS NULL"
        arcpy.management.SelectLayerByAttribute(output_feature_class, "NEW_SELECTION", where_clause)

        move_features_to_location(output_feature_class, y, x, old_sign_code)  # y as latitude, x as longitude
        if sign_code == 'Design' or sign_code == 'NoCode':
            continue 
            
        angle = direction_dict.get(direction, 0)
        rotate(output_feature_class, angle)


arcpy.conversion.ExportCAD(
    in_features = output_feature_class,
    Output_Type = "DGN_V8",
    Output_File = f'{input_save_location}\{input_project_name}_signs.dgn',
    Ignore_FileNames = "Ignore_Filenames_in_Tables",
    Append_To_Existing = "Overwrite_Existing_Files",
    Seed_File = r"C:\program files\arcgis\pro\Resources\ArcToolbox\Templates\CAD\template2d_US_Feet.dgn"
)

print("Processing complete.")

No features found for SIGN_CODE: W16-7MPL. Using default no find symbol.
No features found for SIGN_CODE: R10-4BL. Using default no find symbol.
No features found for SIGN_CODE: D9-10AR. Using default no find symbol.
No features found for SIGN_CODE: W16-7MPL. Using default no find symbol.
No features found for SIGN_CODE: W16-7MPL. Using default no find symbol.
No features found for SIGN_CODE: R10-4BR. Using default no find symbol.
No features found for SIGN_CODE: X4-2B. Using default no find symbol.
No features found for SIGN_CODE: W1-8R. Using default no find symbol.
No features found for SIGN_CODE: M1-5A. Using default no find symbol.
No features found for SIGN_CODE: W16-7MPL. Using default no find symbol.
No features found for SIGN_CODE: D1-X4R45. Using default no find symbol.
No features found for SIGN_CODE: R10-4BL. Using default no find symbol.
No features found for SIGN_CODE: I-X1. Using default no find symbol.
No features found for SIGN_CODE: M1-SB9. Using default no find symbo

NameError: name 'output_annotation_class' is not defined

In [ ]:
# Set the 